# Deploying Punctuation and Capitalization Model in RIVA

[Train Adapt Optimize (TAO) Toolkit ](https://developer.nvidia.com/tao-toolkit) provides the capability to export your model in a format that can deployed using NVIDIA [Riva](https://developer.nvidia.com/riva), a highly performant application framework for multi-modal conversational AI services using GPUs.

This tutorial explores taking a .riva model, the result of `tao punctuation_and_capitalization export` command, and leveraging the Riva ServiceMaker framework to aggregate all the necessary artifacts for Riva deployment to a target environment. Once the model is deployed in Riva, you can issue inference requests to the server. We will demonstrate how quick and straightforward this whole process is.

## Learning Objectives

In this notebook, you will learn how to:  
- Use Riva ServiceMaker to take a TAO exported .riva and convert it to .rmir
- Deploy the model(s) locally  on the Riva Server
- Send inference requests from a demo client using Riva API bindings.

## Prerequisites
Before going through the jupyter notebook, please make sure:
- You have access to NVIDIA NGC, and are able to download the Riva Quickstart [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart)
- Have a .riva model file that you wish to deploy. You can obtain this from ``tao <task> export`` (with ``export_format=RIVA``). 

<b>NOTE:</b> Please refer to the tutorial on *Punctuation And Capitalization using Train Adapt Optimize (TAO) Toolkit* for more details on training and exporting a .riva model for punctuation and capitalization task.

## Riva ServiceMaker

Servicemaker is the set of tools that aggregates all the necessary artifacts (models, files, configurations, and user settings) for Riva deployment to a target environment. It has two main components as shown below:

### 1. Riva-build

This step helps build a Riva-ready version of the model. It’s only output is an intermediate format (called a RMIR) of an end to end pipeline for the supported services within Riva. We are taking a ASR QuartzNet Model in consideration<br>

`riva-build` is responsible for the combination of one or more exported models (.riva files) into a single file containing an intermediate format called Riva Model Intermediate Representation (.rmir). This file contains a deployment-agnostic specification of the whole end-to-end pipeline along with all the assets required for the final deployment and inference. Please checkout the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/service-nlp.html#pipeline-configuration) to find out more.

In [ ]:
# IMPORTANT: Set the following variables

# ServiceMaker Docker
RIVA_SM_CONTAINER = "<Riva_Servicemaker_Image>"

# Directory where the .riva model is stored $MODEL_LOC/*.riva
MODEL_LOC = "<path_to_model_directory>"

# Name of the .riva file
MODEL_NAME = "<add model name>"

# Use the same key that .riva model is encrypted with
KEY = "<add encryption key used for trained model>"

In [ ]:
# Pull the ServiceMaker Image
!docker pull $RIVA_SM_CONTAINER

In [ ]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
!docker run --rm --gpus all -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
    riva-build punctuation -f /data/punct-capit.rmir:$KEY /data/$MODEL_NAME:$KEY

`NOTE:` Above, punct-capit-model.rmir is the punctuation and capitalization model obtained from `tao punctuation_and_capitalization export`

### 2. Riva-deploy

The deployment tool takes as input one or more Riva Model Intermediate Representation (RMIR) files and a target model repository directory. It creates an ensemble configuration specifying the pipeline for the execution and finally writes all those assets to the output model repository directory.

In [ ]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
!docker run --rm --gpus all -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
     riva-deploy -f /data/punct-capit.rmir:$KEY /data/models

## Start Riva Server

Once the model repository is generated, we are ready to start the Riva server. From this step onwards you need to download the Riva QuickStart Resource from NGC. 

In [ ]:
### Set the path to Riva directory
RIVA_DIR = <path_to_riva_quickstart>

Next, we modify ``config.sh`` to enable relevant Riva services (nlp for Punctuation & Capitalization model), provide the encryption key, and path to the model repository (``riva_model_loc``) generated in the previous step among other configurations.

Pretrained versions of models specified in models_asr/nlp/tts are fetched from NGC. Since we are using our custom model, we can comment it in models_nlp (and any others that are not relevant to our use case). 

#### config.sh snipet
```
# Enable or Disable Riva Services 
service_enabled_asr=false                                 ## MAKE CHANGES HERE
service_enabled_nlp=true                                  ## MAKE CHANGES HERE
service_enabled_tts=false                                 ## MAKE CHANGES HERE

# Specify one or more GPUs to use
# specifying more than one GPU is currently an experimental feature, and may result in undefined behaviours.
gpus_to_use="device=0"

# Specify the encryption key to use to deploy models
MODEL_DEPLOY_KEY="tlt_encode"                             ## Set the model encryption key

# Locations to use for storing models artifacts
...
riva_model_loc="<add path>"                              ## Replace with MODEL_LOC

# The default RMIRs are downloaded from NGC by default in the above $riva_rmir_loc directory
# If you'd like to skip the download from NGC and use the existing RMIRs in the $riva_rmir_loc
# then set the below $use_existing_rmirs flag to true.
...
use_existing_rmirs=true                                  ## Set to True
```

In [ ]:
# Ensure you have permission to execute these scripts.
!cd $RIVA_DIR && chmod +x ./riva_init.sh && chmod +x ./riva_start.sh

In [ ]:
# Run Riva Init. This will fetch the containers/models
# YOU CAN SKIP THIS STEP IF YOU DID RIVA DEPLOY
!cd $RIVA_DIR && ./riva_init.sh config.sh

In [ ]:
 # Run Riva Start. This will deploy your model(s).
!cd $RIVA_DIR && ./riva_start.sh config.sh

## Run Inference
Once the Riva server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Riva Python API bindings for client. This is available as a pip .whl with the QuickStart.

In [ ]:
# IMPORTANT: Set the name of the whl file
RIVA_API_WHL = "<add riva api .whl file name>"

In [ ]:
# Install client API bindings
!cd $RIVA_DIR && pip install $RIVA_API_WHL

Run the following sample code from within the client docker container:

In [ ]:
import grpc
import argparse
import os
import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv

class BertPunctuatorClient(object):
    def __init__(self, grpc_server, model_name="riva_punctuation"):
        # generate the correct model based on precision and whether or not ensemble is used
        print("Using model: {}".format(model_name))
        self.model_name = model_name
        self.channel = grpc.insecure_channel(grpc_server)
        self.riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(self.channel)

        self.has_bos = True
        self.has_eos = False

    def run(self, input_strings):
        if isinstance(input_strings, str):
            # user probably passed a single string instead of a list/iterable
            input_strings = [input_strings]

        request = rnlp.TextTransformRequest()
        request.model.model_name = self.model_name
        for q in input_strings:
            request.text.append(q)
        response = self.riva_nlp.TransformText(request)

        return response.text[0]

def run_punct_capit(server,model,query):
    print("Client app to test punctuation and capitalization on Riva")
    client = BertPunctuatorClient(server, model_name=model)
    result = client.run(query)
    print(result)

In [ ]:
run_punct_capit(server="localhost:50051",
                model="riva_punctuation",
                query="how are you doing")

You can stop all docker container before shutting down the jupyter kernel.

In [ ]:
!docker stop $(docker ps -a -q)